In [ ]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

# PicoSat

[PicoSat](https://fmv.jku.at/picosat/) is a SAT solver by [Armine Biere](https://cca.informatik.uni-freiburg.de/biere/).
A [Python API](https://github.com/conda/pycosat) to this solver is available as [pycosat](https://pypi.org/project/pycosat/).  We can install it via `pip`.

In [ ]:
pip install pycosat

In [ ]:
import pycosat

In *PicoSat* propositional variables are represented as follows:
* A propositional variable $p$ is represented as a positive natural number.
* If $n$ is the natural number representing $p$, then $\neg p$ is represented as the integer $-n$.
* A clause is represented as a list of integers.
* A formula in CNF is represented as a list of clauses and hence as a list of list of integers.

For example, if the propositional variables $p$, $q$, and $r$ are represented as the natural numbers
$1$, $2$, and $3$ respectively, then 
* $p \vee \neg q \vee r$ is represented as the list `[1, -2, 3]`,
* $\neg p \vee q \vee \neg r$ is represented as the list `[-1, 2, -3]`,
* $\neg p \vee \neg q \vee r$ is represented as the list `[-1, -2, 3]`, and
* $p \vee q \vee \neg r$ is represented as the list `[1, 2, -3]`.

Finally, the formula 
$$ (p \vee \neg q \vee r) \wedge (\neg p \vee q \vee \neg r) \wedge (\neg p \vee \neg q \vee r) \wedge (p \vee q \vee \neg r), $$
which is in conjunctive normal form, is represented as follows:
```
   [ [1, -2, 3], [-1, 2, -3], [-1, -2, 3], [1, 2, -3] ]
```

In [ ]:
f = [ [1, -2, 3], [-1, 2, -3], [-1, -2, 3], [1, 2, -3] ]

In order to check whether this formula is satifiable, we can use the method `pycosat.solve` as follows:

In [ ]:
pycosat.solve(f)

This shows that the formula `f` is satisfiable and that the  propositional valuation 
$$ \mathcal{I} = \{ p \mapsto \texttt{False}, p \mapsto \texttt{False}, p \mapsto \texttt{False} \} $$
is a solution for `f`, i.e. we have
$$ \mathcal{I}(\texttt{f}) = \texttt{True}. $$

## Transforming Clauses into PyCoSat Format

In order to use *PicoSat* for the examples discussed in our lecture, we need a function that transforms a formula that is in conjunctive normal form
into the format of *PicoSat*.  Furthermore, we nee a function that can translate a solution found by *PicoSat* back into our format.

The function `findVariables` takes a set of `Clauses` and returns the set of all propositional variables
occurring in this set.  

In [ ]:
def findVariables(Clauses):
    Variables = set()
    for Clause in Clauses:
        for literal in Clause:
            match literal:
                case ('¬', var): Variables |= { var }
                case var       : Variables |= { var }
    return Variables

The function `numberVariables(Clauses)` takes a set of `Clauses` as input.  It returns two dictionaries:
* The dictionary `Var2Int` maps every propositional variable occurring in `Clauses` to a unique natural number.
* The dictionary `Int2Var` is the mapping that is inverse to the dictionary `Var2Int`.

In [ ]:
def numberVariables(Clauses):
    Variables = findVariables(Clauses)
    count     = 1
    Var2Int   = {}
    Int2Var   = {}
    for variable in Variables:
        Var2Int[variable] = count
        Int2Var[count   ] = variable
        count += 1
    return Var2Int, Int2Var

The function `literal2int` takes a literal and transforms this literal into an integer number,
representing the literal.  If the literal is a negated variable, the integer is negative, else it is positive.
`Var2Int` is a dictionary mapping propositional variables to integers.

In [ ]:
def literal2int(literal, Var2Int):
    match literal:
        case ('¬', var): return -Var2Int[var]
        case var       : return  Var2Int[var]

The function `clause2pyco(Clause, Var2Int)` transforms a set of literals into a list of integers.
`Var2Int` is a dictionary mapping the propositional variables to integers.

In [ ]:
def clause2pyco(Clause, Var2Int):
    return [literal2int(literal, Var2Int) for literal in Clause]

The function `clauses2pyco(Clauses, Var2Int)` transforms a set of `Clauses` into a list of lists of integers.
`Var2Int` is a dictionary mapping the propositional variables to integers.

In [ ]:
def clauses2pyco(Clauses, Var2Int):
    return [clause2pyco(clause, Var2Int) for clause in Clauses ]

The function `int2var(Numbers, Int2Var)` takes a list of numbers representing a set of literals
and returns the associated list of literals.

In [ ]:
def int2var(Numbers, Int2Var):
    Result = set()
    for n in Numbers:
        if n > 0:
            Result |= {frozenset({Int2Var[n]})}
        else: 
            Result |= {frozenset({ ('¬', Int2Var[-n]) })}
    return Result

## Solving the N-Queens-Problem

In [ ]:
%%capture
%run 08-N-Queens.ipynb

The function $\texttt{queens}(n)$ solves the *n queens problem*.

In [ ]:
def queens(n):
    "Solve the n queens problem."
    Clauses  = allClauses(n)
    V2I, I2V = numberVariables(Clauses)
    Clauses  = clauses2pyco(Clauses, V2I)
    Solution = pycosat.solve(Clauses)
    if Solution:
        return int2var(Solution, I2V)
    else:
        print(f'The problem is not solvable for {n} queens!')

In [ ]:
%%time
Solution = queens(16)

Note that this is more than a 100 times faster than our Python implementation of the Davis-Putnam algorithm.
There are two reasons for this:
* The version of the Davis-Putnam algorithm that is implemented in *PycoSat* is much more refined
  than our naive implementation.
* The programming language `C` is much faster than Python.

In [ ]:
show_solution(Solution)